In [1]:
# Install Ultralytics
!pip install ultralytics

# Verify dataset structure
print("Listing /kaggle/input:")
!ls /kaggle/input
print("\nListing /kaggle/input/tennis-object-detection:")
!ls /kaggle/input/tennis-object-detection
print("\nListing /kaggle/input/tennis-object-detection/tennis-object-detection:")
!ls /kaggle/input/tennis-object-detection/tennis-object-detection
print("\nListing /kaggle/input/tennis-object-detection/tennis-object-detection/train:")
!ls /kaggle/input/tennis-object-detection/tennis-object-detection/train
print("\nListing /kaggle/input/tennis-object-detection/tennis-object-detection/valid:")
!ls /kaggle/input/tennis-object-detection/tennis-object-detection/valid

# Copy data.yaml to a writable directory and update it
import os
import shutil

# Copy data.yaml to /kaggle/working/
shutil.copy("/kaggle/input/tennis-object-detection/tennis-object-detection/data.yaml", "/kaggle/working/data.yaml")

# Update data.yaml with the correct paths
yaml_content = """train: /kaggle/input/tennis-object-detection/tennis-object-detection/train/images
val: /kaggle/input/tennis-object-detection/tennis-object-detection/valid/images
test: /kaggle/input/tennis-object-detection/tennis-object-detection/test/images
nc: 2
names: ['ball', 'player']
"""
with open("/kaggle/working/data.yaml", "w") as f:
    f.write(yaml_content)

# Step 1: Load the existing best model weights and save as .pkl
from ultralytics import YOLO
import pickle
import torch

# Load the existing best model weights
# Adjust the path if your best.pt is in a different location
best_model_path = "/kaggle/input/best1/pytorch/default/1/best (1).pt"
best_model = YOLO(best_model_path)
print(f"Best model loaded from {best_model_path}")

# Extract the best model state dictionary
best_model_state_dict = best_model.model.state_dict()

# Save the best weights to .pkl
# Save the best weights to .pkl
with open("/kaggle/working/tennis_player_model_best_weights.pkl", "wb") as f:
    pickle.dump(best_model_state_dict, f)

print("Best model weights saved as /kaggle/working/tennis_player_model_best_weights.pkl")

# Step 2: Load weights from .pkl and validate
# Initialize a new YOLO model directly from best.pt (already configured for nc=2)
new_model = YOLO(best_model_path)  # Load from best.pt to avoid configuration mismatch

# Load the saved state dictionary from .pkl
with open("/kaggle/working/tennis_player_model_best_weights.pkl", "rb") as f:
    loaded_state_dict = pickle.load(f)

# Load the weights into the new model
new_model.model.load_state_dict(loaded_state_dict)
new_model.model.eval()  # Set to evaluation mode

# Step 3: Validate using the loaded weights
# Use split='val' to match the 'val' key in data.yaml
val_results = new_model.val(data="/kaggle/working/data.yaml", split='val', imgsz=1280)

# Step 4: Calculate Accuracy, Precision, Recall, and F1 Score
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

# Extract metrics from validation results
precision = val_results.results_dict['metrics/precision(B)']
recall = val_results.results_dict['metrics/recall(B)']
mAP50 = val_results.results_dict['metrics/mAP50(B)']

# Compute F1 score
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Approximate accuracy for object detection
if precision > 0 and recall > 0:
    accuracy = (precision * recall) / (precision + recall - (precision * recall))
else:
    accuracy = 0

print(f"Validation Metrics (using loaded .pkl weights):")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"mAP@50: {mAP50:.4f}")
print(f"Approximate Accuracy: {accuracy:.4f}")

# Test on an image (optional)
results = new_model.predict("/kaggle/input/tennis-object-detection/tennis-object-detection/valid/images/frame_1035_jpg.rf.c0df7a1166487c5a480f21a1af403be9.jpg", imgsz=1280)
results[0].save("/kaggle/working/output.jpg")
print("Test image saved as /kaggle/working/output.jpg")

# Download model weights
from IPython.display import FileLink
FileLink(r'/kaggle/working/runs/train/tennis_player_model/weights/best.pt')
FileLink(r'/kaggle/working/tennis_player_model_best_weights.pkl')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 10.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existin

100%|██████████| 755k/755k [00:00<00:00, 40.3MB/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 32.1±5.3 MB/s, size: 239.5 KB)



val: Scanning /kaggle/input/tennis-object-detection/tennis-object-detection/valid/labels... 171 images, 0 backgrounds, 0 corrupt: 100%|██████████| 171/171 [00:01<00:00, 155.75it/s]

WARNING ⚠️ val: Cache directory /kaggle/input/tennis-object-detection/tennis-object-detection/valid is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:07<00:00,  1.39it/s]


                   all        171        501      0.897      0.868      0.872      0.519
                  ball        154        158      0.855      0.753      0.759      0.299
                player        171        343      0.939      0.983      0.985      0.739


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 7.3ms preprocess, 17.9ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/val
Validation Metrics (using loaded .pkl weights):
Precision: 0.8970
Recall: 0.8678
F1 Score: 0.8822
mAP@50: 0.8721
Approximate Accuracy: 0.7892

image 1/1 /kaggle/input/tennis-object-detection/tennis-object-detection/valid/images/frame_1035_jpg.rf.c0df7a1166487c5a480f21a1af403be9.jpg: 1280x1280 1 ball, 2 players, 20.0ms
Speed: 10.5ms preprocess, 20.0ms inference, 3.3ms postprocess per image at shape (1, 3, 1280, 1280)
Test image saved as /kaggle/working/output.jpg


/kaggle/working/tennis_player_model_best_weights.pkl